# 🔥 VTrackAI Studio - Kaggle E2E GPU Setup

**Prerequisites:**
1. Settings → Accelerator → **GPU P100** or **T4 x2**
2. Add-ons → Secrets → Add `HF_TOKEN` (from huggingface.co/settings/tokens)
3. Enable **Internet** in the sidebar
4. Run all cells (Ctrl+Shift+Enter)

---

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 1: SETUP (Run once per session, ~3-5 min)
# ═══════════════════════════════════════════════════════════════════════════════
import torch
import os
import sys
import subprocess

# Define base directory (Kaggle uses /kaggle/working)
BASE_DIR = "/kaggle/working"
os.chdir(BASE_DIR)

# GPU Check
print("="*60)
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"📊 VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("❌ NO GPU! Go to Settings → Accelerator → GPU P100 or T4")
    sys.exit(1)
print("="*60)

# Clean start
subprocess.run(["rm", "-rf", f"{BASE_DIR}/vtrack-ai-studio"])

# Install dependencies
print("\n📦 Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "torch", "torchvision", "--index-url", "https://download.pytorch.org/whl/cu121"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "fastapi", "uvicorn[standard]", "python-multipart", "huggingface_hub", "requests", "decord"])
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "numpy==1.26.4"])

# Node.js 18 via nodeenv
print("\n📦 Installing Node.js 18...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "nodeenv"])
subprocess.run(["nodeenv", "--node=18.20.0", f"{BASE_DIR}/node_env", "--force", "-q"])

# Set up node path
NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

# Verify node installation
subprocess.run([f"{NODE_BIN}/node", "--version"])
subprocess.run([f"{NODE_BIN}/npm", "--version"])

# Install localtunnel globally
print("\n📦 Installing localtunnel...")
subprocess.run([f"{NODE_BIN}/npm", "install", "-g", "localtunnel", "--silent"])

# Clone VTrackAI
print("\n📦 Cloning VTrackAI...")
subprocess.run(["git", "clone", "-q", "https://github.com/Amitro123/vtrack-ai-studio", f"{BASE_DIR}/vtrack-ai-studio"])
subprocess.run([sys.executable, "-m", "pip", "install", "-r", f"{BASE_DIR}/vtrack-ai-studio/backend/requirements.txt", "-q"], stderr=subprocess.DEVNULL)

# Clone SAM3 INTO backend/sam3 (where server.py expects it)
print("\n📦 Cloning SAM3 into backend/sam3...")
BACKEND_DIR = f"{BASE_DIR}/vtrack-ai-studio/backend"
os.chdir(BACKEND_DIR)
subprocess.run(["rm", "-rf", "./sam3"])
subprocess.run(["git", "clone", "-q", "https://github.com/facebookresearch/sam3", "./sam3"])
subprocess.run([sys.executable, "-m", "pip", "install", "-e", "./sam3", "-q"], stderr=subprocess.DEVNULL)
print("✅ SAM3 installed in backend/sam3")

# HuggingFace login + checkpoint
print("\n🔐 Downloading SAM3 checkpoint...")
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login, hf_hub_download

user_secrets = UserSecretsClient()
HF_TOKEN = user_secrets.get_secret("HF_TOKEN")
login(token=HF_TOKEN, add_to_git_credential=False)

# Create checkpoint directories
checkpoint_dir = f"{BACKEND_DIR}/checkpoints/sam3"
checkpoints_subdir = f"{checkpoint_dir}/checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(checkpoints_subdir, exist_ok=True)

# Download checkpoint
checkpoint_path = hf_hub_download(
    repo_id="facebook/sam3",
    filename="sam3.pt",
    local_dir=checkpoint_dir,
    token=HF_TOKEN,
)
print(f"✅ Downloaded: {checkpoint_path}")

# Create symlinks for all checkpoint name variants SAM3 might look for
sam3_pt = f"{checkpoint_dir}/sam3.pt"
symlinks = [
    f"{checkpoint_dir}/sam3_hiera_large.pt",
    f"{checkpoints_subdir}/sam3_hiera_tiny.pt",
    f"{checkpoints_subdir}/sam3_hiera_large.pt",
]

for dst in symlinks:
    try:
        if os.path.islink(dst) or os.path.exists(dst):
            os.remove(dst)
        os.symlink(sam3_pt, dst)
        print(f"✅ Symlink: {os.path.basename(dst)}")
    except OSError as e:
        print(f"❌ Symlink failed: {dst} - {e}")

# Go back to base dir
os.chdir(BASE_DIR)

print("\n" + "="*60)
print("✅ SETUP COMPLETE! Run Cell 2 to start backend.")
print("="*60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 2: START BACKEND + TUNNEL (FastAPI on port 8000)
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess
import time
import requests
import os
import re
import urllib.request

BASE_DIR = "/kaggle/working"
BACKEND_DIR = f"{BASE_DIR}/vtrack-ai-studio/backend"
NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

# Get public IP for tunnel password
print("🔍 Getting public IP address...")
try:
    PUBLIC_IP = urllib.request.urlopen('https://api.ipify.org').read().decode('utf8')
    print(f"✅ Public IP: {PUBLIC_IP}")
except Exception:
    PUBLIC_IP = "(could not detect)"

# Kill any existing processes
subprocess.run(["pkill", "-9", "-f", "uvicorn"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "lt"], stderr=subprocess.DEVNULL)
time.sleep(2)

# Verify SAM3 is installed
print("\n🔍 Checking SAM3 installation...")
sam3_path = f"{BACKEND_DIR}/sam3"
if os.path.exists(sam3_path):
    print(f"✅ SAM3 found at {sam3_path}")
else:
    print(f"❌ SAM3 NOT found at {sam3_path}")

# Check checkpoints
checkpoint_dir = f"{BACKEND_DIR}/checkpoints/sam3"
if os.path.exists(f"{checkpoint_dir}/sam3.pt"):
    print(f"✅ Checkpoint found")
else:
    print(f"❌ Checkpoint NOT found")

# Start backend in background
print("\n🚀 Starting FastAPI backend...")
log_file = open(f"{BASE_DIR}/backend.log", "w")
proc = subprocess.Popen(
    ["uvicorn", "server:app", "--host", "0.0.0.0", "--port", "8000"],
    stdout=log_file,
    stderr=subprocess.STDOUT,
    cwd=BACKEND_DIR,
)

# Wait for backend
backend_ready = False
for i in range(30):
    if proc.poll() is not None:
        print("❌ Backend crashed! Logs:")
        with open(f"{BASE_DIR}/backend.log") as f:
            print(f.read())
        break
    try:
        r = requests.get("http://localhost:8000/api/health", timeout=3)
        health = r.json()
        print(f"✅ Backend running")
        print(f"   SAM3: {'✅ Available' if health.get('sam3_available') else '❌ Not loaded'}")
        print(f"   Device: {health.get('device', 'unknown')}")
        backend_ready = True
        break
    except Exception:
        print(f"⏳ Waiting for backend... ({i+1}/30)")
        time.sleep(2)

if not backend_ready:
    print("❌ Backend failed to start. Logs:")
    with open(f"{BASE_DIR}/backend.log") as f:
        print(f.read()[-2000:])
else:
    # Start BACKEND tunnel
    print("\n🌐 Creating backend tunnel...")
    backend_tunnel_log = open(f"{BASE_DIR}/backend_tunnel.log", "w")
    backend_tunnel = subprocess.Popen(
        [f"{NODE_BIN}/lt", "--port", "8000"],
        stdout=backend_tunnel_log,
        stderr=subprocess.STDOUT,
        env={**os.environ}
    )
    time.sleep(8)
    
    # Get backend tunnel URL
    with open(f"{BASE_DIR}/backend_tunnel.log") as f:
        log = f.read()
    backend_urls = re.findall(r'https://[\w-]+\.loca\.lt', log)
    
    if backend_urls:
        BACKEND_URL = backend_urls[0]
        # Save for frontend to use
        with open(f"{BASE_DIR}/backend_url.txt", "w") as f:
            f.write(BACKEND_URL)
        print(f"\n" + "="*60)
        print(f"✅ Backend tunnel: {BACKEND_URL}")
        print(f"🔑 Tunnel password: {PUBLIC_IP}")
        print("="*60)
        print("\n✅ Run Cell 3 to start frontend.")
    else:
        print("❌ Failed to get backend tunnel URL")
        print(log)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 3: START FRONTEND + TUNNEL (Vite on port 4173)
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess
import time
import os
import re
import urllib.request

BASE_DIR = "/kaggle/working"
NODE_BIN = f"{BASE_DIR}/node_env/bin"
os.environ["PATH"] = f"{NODE_BIN}:{os.environ['PATH']}"

# Get public IP for tunnel password
try:
    PUBLIC_IP = urllib.request.urlopen('https://api.ipify.org').read().decode('utf8')
except Exception:
    PUBLIC_IP = "(could not detect)"

# Read backend tunnel URL
try:
    with open(f"{BASE_DIR}/backend_url.txt") as f:
        BACKEND_URL = f.read().strip()
    print(f"📡 Backend URL: {BACKEND_URL}")
except Exception:
    print("❌ Backend URL not found. Run Cell 2 first!")
    BACKEND_URL = "http://localhost:8000"  # Fallback

# Kill any existing frontend
subprocess.run(["pkill", "-9", "-f", "vite"], stderr=subprocess.DEVNULL)
time.sleep(1)

# Install frontend dependencies
print("\n🚀 Installing frontend dependencies...")
os.chdir(f"{BASE_DIR}/vtrack-ai-studio")

# Write .env file with BACKEND TUNNEL URL (not localhost!)
with open(".env", "w") as f:
    f.write(f"VITE_API_URL={BACKEND_URL}\n")
print(f"✅ Set VITE_API_URL={BACKEND_URL}")

subprocess.run([f"{NODE_BIN}/npm", "install", "--silent"], stderr=subprocess.DEVNULL)

# Start Vite dev server
print("\n🚀 Starting Vite frontend...")
vite_log = open(f"{BASE_DIR}/vite.log", "w")
vite_proc = subprocess.Popen(
    [f"{NODE_BIN}/npm", "run", "dev", "--", "--host", "0.0.0.0", "--port", "4173"],
    stdout=vite_log,
    stderr=subprocess.STDOUT,
    cwd=f"{BASE_DIR}/vtrack-ai-studio",
    env={**os.environ}
)

print("⏳ Waiting for Vite to start...")
time.sleep(15)

# Start FRONTEND tunnel
print("\n🌐 Creating frontend tunnel...")
frontend_tunnel_log = open(f"{BASE_DIR}/frontend_tunnel.log", "w")
frontend_tunnel = subprocess.Popen(
    [f"{NODE_BIN}/lt", "--port", "4173"],
    stdout=frontend_tunnel_log,
    stderr=subprocess.STDOUT,
    env={**os.environ}
)

time.sleep(8)

# Get frontend tunnel URL
with open(f"{BASE_DIR}/frontend_tunnel.log") as f:
    log = f.read()
frontend_urls = re.findall(r'https://[\w-]+\.loca\.lt', log)

if frontend_urls:
    FRONTEND_URL = frontend_urls[0]
    print("\n" + "="*60)
    print("🎉 VTrackAI Studio is LIVE!")
    print(f"\n   👉 FRONTEND: {FRONTEND_URL}")
    print(f"   📡 BACKEND:  {BACKEND_URL}")
    print(f"\n   🔑 TUNNEL PASSWORD: {PUBLIC_IP}")
    print("\n   ⚠️  You need to enter the password on BOTH tunnel pages!")
    print("      1. First visit the BACKEND URL and enter the password")
    print("      2. Then visit the FRONTEND URL and enter the password")
    print("="*60)
else:
    print("❌ Failed to get frontend tunnel URL")
    print(log)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 4: GET ALL URLS & PASSWORD (Run if needed)
# ═══════════════════════════════════════════════════════════════════════════════
import re
import urllib.request

BASE_DIR = "/kaggle/working"

# Get public IP
try:
    PUBLIC_IP = urllib.request.urlopen('https://api.ipify.org').read().decode('utf8')
except Exception:
    PUBLIC_IP = "(could not detect)"

print("=" * 60)
print("=== VTrackAI Access Info ===")
print("=" * 60)

# Backend URL
try:
    with open(f"{BASE_DIR}/backend_tunnel.log") as f:
        log = f.read()
    urls = re.findall(r'https://[\w-]+\.loca\.lt', log)
    if urls:
        print(f"\n📡 BACKEND: {urls[0]}")
except Exception:
    print("\n📡 BACKEND: Not found")

# Frontend URL
try:
    with open(f"{BASE_DIR}/frontend_tunnel.log") as f:
        log = f.read()
    urls = re.findall(r'https://[\w-]+\.loca\.lt', log)
    if urls:
        print(f"👉 FRONTEND: {urls[0]}")
except Exception:
    print("👉 FRONTEND: Not found")

print(f"\n🔑 TUNNEL PASSWORD: {PUBLIC_IP}")
print("\n⚠️  Enter password on BOTH URLs before using the app!")
print("=" * 60)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 5: DEBUG / LOGS (Run if needed)
# ═══════════════════════════════════════════════════════════════════════════════
import os
import subprocess

BASE_DIR = "/kaggle/working"

print("=== SAM3 Check ===")
sam3_path = f"{BASE_DIR}/vtrack-ai-studio/backend/sam3"
print(f"SAM3 exists: {os.path.exists(sam3_path)}")
if os.path.exists(sam3_path):
    contents = os.listdir(sam3_path)
    print(f"SAM3 contents: {contents[:10]}...")

print("\n=== Checkpoint Check ===")
checkpoint_dir = f"{BASE_DIR}/vtrack-ai-studio/backend/checkpoints/sam3"
if os.path.exists(checkpoint_dir):
    for f in os.listdir(checkpoint_dir):
        full_path = os.path.join(checkpoint_dir, f)
        if os.path.islink(full_path):
            print(f"  {f} -> {os.readlink(full_path)}")
        else:
            size = os.path.getsize(full_path) / 1e6 if os.path.isfile(full_path) else 0
            print(f"  {f} ({size:.1f} MB)")

print("\n=== Backend Logs ===")
try:
    with open(f"{BASE_DIR}/backend.log") as f:
        lines = f.readlines()[-20:]
        print("".join(lines))
except Exception:
    print("No backend log")

print("\n=== Process Status ===")
result = subprocess.run(["ps", "aux"], capture_output=True, text=True)
for line in result.stdout.split("\n"):
    if any(x in line for x in ["uvicorn", "node", "lt"]) and "grep" not in line:
        print(line)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CELL 6: CLEANUP (Run when done)
# ═══════════════════════════════════════════════════════════════════════════════
import subprocess

print("🧹 Cleaning up...")

# Kill processes
subprocess.run(["pkill", "-9", "-f", "uvicorn"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "node"], stderr=subprocess.DEVNULL)
subprocess.run(["pkill", "-9", "-f", "lt"], stderr=subprocess.DEVNULL)

print("✅ Cleanup complete. All services stopped.")